In [1]:
# !pip install lightning
# !pip install protobuf
# !pip install transformers
# !pip install sentencepiece

In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import pandas as pd
from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import train_test_split
import os
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [5]:
class SentenceSimilarityModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-5):
        super(SentenceSimilarityModel, self).__init__()
        self.berta = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',ignore_mismatched_sizes=True,num_labels=1,output_hidden_states= True)


        self.learning_rate = learning_rate

    def forward(self, input_ids,attention_mask):
        outputs = self.berta(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        return torch.sigmoid(logits)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, score = batch
        logits = self(input_ids,attention_mask)


        loss = F.mse_loss(logits, score)
        spearman = spearmanr(logits.detach().cpu().numpy(),score.detach().cpu().numpy()).statistic

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_spearman", spearman, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, score = batch
        logits = self(input_ids, attention_mask)


        loss = F.mse_loss(logits, score)

        spearman = spearmanr(logits.detach().cpu().numpy(),score.detach().cpu().numpy()).statistic

        self.log("val_loss", loss,on_epoch = True, prog_bar = True)
        self.log("val_spearman",spearman,on_epoch = True, prog_bar = True)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)



class str_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract the features and target from the DataFrame
        # Adjust this based on your DataFrame structure
        features = self.dataframe['input'].loc[idx]
        token = self.tokenizer(features, return_tensors='pt', truncation='longest_first', padding='max_length',max_length=265)

        input_ids = token['input_ids'].squeeze()
        attention_mask = token['attention_mask'].squeeze()

        score = self.dataframe['Score'].loc[idx]

        score = torch.tensor(score, dtype=torch.float32).unsqueeze(dim=0)  # Adjust the dtype as needed

        return input_ids, attention_mask, score



class STR_DataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=16):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size,num_workers = 2, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,num_workers = 2)


In [6]:
if __name__ == '__main__':
    train_data = pd.read_csv('/content/eng_train.csv')
    sep = '[SEP]'
    train_data['input'] = train_data.apply(lambda row : row['Text'].replace('\n',sep),axis = 1)

    train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

    train_dataset = str_dataset(train_df.reset_index(drop=True))
    val_dataset = str_dataset(val_df.reset_index(drop=True))

    str_datamodule = STR_DataModule(train_dataset=train_dataset, val_dataset= val_dataset, batch_size = 32)


    model = SentenceSimilarityModel()

    trainer = pl.Trainer(max_epochs=100,precision="16-mixed",callbacks=[EarlyStopping(monitor="val_loss", patience=3, mode="min")],accelerator='gpu')

    tuner = pl.tuner.Tuner(trainer)
    lr_finder = tuner.lr_find(model, datamodule = str_datamodule)

    # Pick point based on plot, or get suggestion
    new_lr = lr_finder.suggestion()

    # # update hparams of the model
    model.lr = new_lr


    trainer.fit(model, datamodule=str_datamodule)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
INFO:pytorch_lightning.tuner.lr_finder:LR finder stopped early after 78 steps due to diverging loss.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 1.7378008287493761e-06
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_068e7220-c815-4770-b7f1-9c1c271dd55d.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr_find_068e7220-c815-4770-b7f1-9c1c271dd55d.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                                | Params
--------------------------------------------------------------
0 | berta | DistilBertForSequenceClassification | 67

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]